In [1]:
from pathlib import Path

import pandas as pd
import scanpy as sc
import scrublet as scr

sc.settings.verbosity = 4
sc.settings.set_figure_params(80)
sc.settings.file_format_figures = 'pdf'
sc.settings.savefigs = False
# use_first_n_samples = 0
# full_sparse = False

# Basic QC workflow

In [2]:
adatas: list[sc.AnnData] = []
data_names = ['achinoam_LF', 'achinoam_control', 'khalisi_LF', 'khalisi_control']

# ...etc - Folders with matrix, features and barcodes gz files.
base_path = Path('../../data/single_cell/animals/')
paths = ['komodo_achinoam/LF',
         'komodo_achinoam/control',
         'komodo_khalisi/LF',
         'komodo_khalisi/control']
for path in paths:
    adata = sc.read_10x_mtx(base_path / path,
                            var_names='gene_symbols',  # use gene symbols for the variable names (variables-axis index)
                            cache=True)
    sc.logging.print_memory_usage()
    print(adata.shape)
    adatas.append(adata)

# adata.obs
# adata.var
# adata.X


    reading ..\..\data\single_cell\animals\komodo_achinoam\LF\matrix.mtx.gz
... writing an h5ad cache file to speedup reading next time
Memory usage: current 0.38 GB, difference +0.38 GB
(6753, 18772)
    reading ..\..\data\single_cell\animals\komodo_achinoam\control\matrix.mtx.gz
... writing an h5ad cache file to speedup reading next time
Memory usage: current 0.53 GB, difference +0.15 GB
(8618, 18772)
    reading ..\..\data\single_cell\animals\komodo_khalisi\LF\matrix.mtx.gz
... writing an h5ad cache file to speedup reading next time
Memory usage: current 0.75 GB, difference +0.21 GB
(9021, 18772)
    reading ..\..\data\single_cell\animals\komodo_khalisi\control\matrix.mtx.gz
... writing an h5ad cache file to speedup reading next time
Memory usage: current 0.66 GB, difference -0.08 GB
(4970, 18772)


In [3]:
adata = adatas[0].concatenate(adatas[1:], batch_categories=data_names)
adata

c:\Python\Python310\lib\site-packages\anndata\_core\anndata.py:1763: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(


AnnData object with n_obs × n_vars = 29362 × 18772
    obs: 'batch'
    var: 'gene_ids', 'feature_types'

In [4]:
genes = pd.read_csv('../../data/single_cell/eggnog_combined.csv')
genes.dropna(subset=['komodo gene id'], inplace=True)

genes_d = dict(
    zip(genes['komodo gene id'].astype('str'),
        genes['eggnog_name'].astype('str'))
)

dict_multi = {}
dict_uni = {}
for gene_id in genes_d.items():
    if len(gene_id.split(',')) != 1:
        dict_multi[gene_id] = genes_d[gene_id]
    else:
        dict_uni[gene_id] = genes_d[gene_id]

multi_result_dict = {}

for key_str, value in dict_multi.items():
    # Convert the string key to a set
    key_set = set(key_str.strip("{}").replace("'", "").split(", "))

    # Iterate through the elements in the set and create individual keys
    for element in key_set:
        multi_result_dict[element] = value

genes_dict = {**dict_uni, **multi_result_dict}


AttributeError: 'tuple' object has no attribute 'split'

In [ ]:
adata.var.set_index('gene_ids', inplace=True)
adata.var['gene_ids'] = adata.var.index

adata.var.drop(columns='feature_types', inplace=True)
adata.var.rename(genes_dict, inplace=True)


In [ ]:
adata.var_names_make_unique(join='-')


### Genes and cells filtration 


Show those genes that yield the highest fraction of counts in each single cell, across all cells.

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)


Basic filtering

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
print(adata.shape)


### Mitochondrial QC and general measures

Check if genes are annotated as mt by running:
- GENES= list(adata.var.index[adata.var.index.str.startswith('mt-'.upper())])
- GENES

In case the genes are not annotated as 'MT-'' (Like in bats), run:

- dict_replace = {'COX1':'MT-COX1','COX2':'MT-COX2'...etc}
- adata.var.rename(dict_replace, inplace = True)

In [ ]:
dict_replace = {'COX1': 'MT-COX1',
                'COX2': 'MT-COX2',
                'COX3': 'MT-COX3',
                'ND1': 'MT-ND1',
                'ND2': 'MT-ND2',
                'ND3': 'MT-ND3',
                'ND4': 'MT-ND4',
                'ND5': 'MT-ND5',
                'ND6': 'MT-ND6',
                'ND4L': 'MT-ND4L',
                'ATP6': 'MT-ATP6',
                'ATP8': 'MT-ATP8',
                'CYTB': 'MT-CYTB'}
adata.var.rename(dict_replace, inplace=True)
# ONLY MT-ATP6, MT-ND1, MT-ND4L


In [ ]:
adata.var[adata.var_names.str.startswith('MT-')]


In [ ]:
adata.var['MT'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'MT'
sc.pp.calculate_qc_metrics(adata,
                           qc_vars=['MT'],
                           percent_top=None,
                           log1p=False,
                           inplace=True)


A violin plot of some of the computed quality measures:

- the number of genes expressed in the count matrix
- the total counts per cell
- the percentage of counts in mitochondrial genes

In [ ]:
sc.pp.calculate_qc_metrics(adata,
                           percent_top=None,
                           log1p=False,
                           inplace=True)


In [ ]:
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color='batch')
sc.pl.scatter(adata, x='total_counts', y='pct_counts_MT', color='batch')
sc.pl.scatter(adata, x='total_counts', y='n_genes', color='batch')


Actually do the filtering by slicing the AnnData object - By pct_counts_MT  and by total_counts /n_genes_by_counts or even n_genes

In [ ]:
adata = adata[adata.obs.pct_counts_MT < 10]
adata = adata[adata.obs.total_counts < 40000, :]  # If filtering outliers (<0.1% of cells)


### Doublet analysis and filtering

In [ ]:
def scrub(adatas, adata, adata_names):  # based on raw individual samples.
    print('Before scrublet: ', adata.shape[0])
    doub_index = []
    barcodes = []
    for data, name in zip(adatas, adata_names):
        data.raw = data
        sc.pp.normalize_total(data, target_sum=1e4)
        sc.pp.log1p(data)
        scrub = scr.Scrublet(data.raw.X)
        data.obs['doublet_scores'], data.obs['predicted_doublets'] = scrub.scrub_doublets()
        scrub.plot_histogram()
        print('Doublets' + name + ' :', data.obs[data.obs['doublet_scores'] > 0.25].shape[0])
        barcodes = data.obs[data.obs['doublet_scores'] < 0.25].index.to_list()
        for barcode in barcodes:
            doub_index.append(barcode + '-' + name)

    adata = adata[adata.obs.index.isin(doub_index)]
    print('After scrublet: ', adata.shape[0])
    return adata


In [ ]:
adata = scrub(adatas, adata, adata_names)  # adata_names in the same order as adatas


### Cell cycle scoring

download Cell cycle txt: https://github.com/scverse/scanpy_usage/blob/master/180209_cell_cycle/data/regev_lab_cell_cycle_genes.txt

In [ ]:
cell_cycle_genes = [
    x.strip()
    for x in open(
        r'C:\Users\TzachiHNB5\Desktop\reptiles\komodo\genes_names\regev_lab_cell_cycle_genes.txt'
    )
]

s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]
s_genes = [x for x in s_genes if x in adata.var_names]
g2m_genes = [x for x in g2m_genes if x in adata.var_names]


In [ ]:
cell_cycle_adata = adata.copy()

sc.pp.normalize_per_cell(cell_cycle_adata, counts_per_cell_after=1e4)
sc.pp.log1p(cell_cycle_adata)
sc.pp.scale(cell_cycle_adata)
sc.tl.score_genes_cell_cycle(cell_cycle_adata, s_genes=s_genes, g2m_genes=g2m_genes)
adata_cc_genes = cell_cycle_adata[:, cell_cycle_genes].copy()
sc.tl.pca(adata_cc_genes)
sc.pl.pca_scatter(adata_cc_genes, color='phase')
adata.obs['S_score'] = cell_cycle_adata.obs['S_score'].copy()
adata.obs['G2M_score'] = cell_cycle_adata.obs['G2M_score'].copy()
adata.obs['phase'] = cell_cycle_adata.obs['phase'].copy()


### Saving adata

In [ ]:
adata.var.index.name = 'eggnog'
adata.var


In [ ]:
adata.write('komodo_lf_and_ctrl.h5ad')